In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

In [8]:
all_vendors = pd.read_csv("data/clean/user_day_all_vendors.csv")

/var/folders/59/8g5__6px0xdb5h_5y97sbpw80000gn/T/ipykernel_40952/3648988354.py:1: DtypeWarning: Columns (9,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  all_vendors = pd.read_csv("data/clean/user_day_all_vendors.csv")


In [9]:
whoop_df = all_vendors[
    ['whoop_recovery', 'whoop_strain', 'whoop_hrv', 'whoop_sleep_score']
    ].dropna()

In [10]:
whoop_df.shape

(6907, 4)

In [11]:
whoop_df.describe()

,whoop_recovery,whoop_strain,whoop_hrv,whoop_sleep_score
count,6907.000000,6907.000000,6907.000000,6907.000000
mean,58.952295,9.216604,51.801868,70.026133
std,12.558349,4.415094,16.448210,11.572109
min,16.000000,0.000000,15.000000,29.000000
25%,50.000000,6.060000,40.000000,62.000000
50%,59.000000,9.210000,51.000000,70.000000
75%,67.000000,12.315000,63.000000,78.000000
max,99.000000,21.000000,120.000000,100.000000


In [12]:
baseline_mae = np.mean(
    np.abs(whoop_df['whoop_recovery'] - whoop_df['whoop_recovery'].mean())
)

baseline_mae

np.float64(9.966705605333965)

In [13]:
X = whoop_df[['whoop_strain', 'whoop_hrv', 'whoop_sleep_score']]
y = whoop_df['whoop_recovery']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

lr_mae = mean_absolute_error(y_test, y_pred)
lr_mae

10.028894010616048

- Linear Regression did not perform well as WHOOP recovery is non-linear, threshold-based and interaction-heavy.
- Tree-based model may perform better

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=6,
    random_state=42
)

rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)
rf_mae = mean_absolute_error(y_test, rf_pred)

rf_mae

10.07042716349711

Linear and tree-based models failed to meaningfully outperform a mean baseline, suggesting WHOOP recovery is not well explained by same-day strain, HRV, and sleep metrics alone. This aligns with physiological intuition and highlights the importance of temporal context and latent features in recovery estimation.